In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
!pip install scikit-hts
import hts

import warnings
warnings.simplefilter("ignore")

# settings
plt.style.use('seaborn')
plt.rcParams["figure.figsize"] = (16, 8)

In [ ]:
# def reduce_mem_usage(df):
#     """ iterate through all the columns of a dataframe and modify the data type
#         to reduce memory usage.        
#     """
#     start_mem = df.memory_usage().sum() / 1024**2
#     print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
#     for col in df.columns:
#         col_type = df[col].dtype
        
#         if col_type != object:
#             c_min = df[col].min()
#             c_max = df[col].max()
#             if str(col_type)[:3] == 'int':
#                 if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
#                     df[col] = df[col].astype(np.int8)
#                 elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
#                     df[col] = df[col].astype(np.int16)
#                 elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
#                     df[col] = df[col].astype(np.int32)
#                 elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
#                     df[col] = df[col].astype(np.int64)  
#             else:
#                 if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                     df[col] = df[col].astype(np.float16)
#                 elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                     df[col] = df[col].astype(np.float32)
#                 else:
#                     df[col] = df[col].astype(np.float64)
#         else:
#             df[col] = df[col].astype('category')

#     end_mem = df.memory_usage().sum() / 1024**2
#     print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
#     print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
#     return df


# def import_data(file):
#     """create a dataframe and optimize its memory usage"""
#     df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
#     df = reduce_mem_usage(df)
#     return df

In [ ]:
# calendar = import_data("/kaggle/input/m5-forecasting-accuracy/calendar.csv")
# df = import_data("/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv")
# prices = import_data("/kaggle/input/m5-forecasting-accuracy/sell_prices.csv")
# # df_test = pd.read_csv("../input/m5-forecasting-accuracy/sales_train_evaluation.csv")
# # submission = pd.read_csv("../input/m5-forecasting-accuracy/sample_submission.csv")

In [ ]:
calendar = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/calendar.csv")
df = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv").drop('item_id', 1)
# df = reduce_mem_usage(df)
# prices = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sell_prices.csv")
# df_test = pd.read_csv("../input/m5-forecasting-accuracy/sales_train_evaluation.csv")
# submission = pd.read_csv("../input/m5-forecasting-accuracy/sample_submission.csv")

Categorical  
item_id, dept_id, cat_id **DONE**

Price related  
price_max, price_min, price_std, price_mean, price_norm, price_unique, price_momentum, 
momentum_m, momentum_y **DONE**

Calendar related  
event_name_1, event_name_2, event_type_1, event_type_2, snap_CA, snap_TX, snap_WI,
tm_d, tm_w, tm_m, tm_y, tm_wm, tm_w_end **DONE**

Lag related

Lag only (15 features): sales_lag_28, sales_lag_29, …, sales_lag_42

Rolling only (8 features): rolling_mean_7, rolling_mean_14, rolling_mean_30,
rolling_std_30, rolling_mean_60, rolling_std_60, rolling_mean_180, rolling_std_180

Lag and roll (12 features): rolling_mean_tmp_i, j for i in [1, 7, 14] and j in [7, 14, 30, 60]. These features were king of tricky because they need recursive evaluation during.

Mean encodings  
enc_cat_id_mean, enc_cat_id_std, enc_dept_id_mean, enc_dept_id_std,
enc_item_id_mean, enc_item_id_std

In [ ]:
import datetime as dt
calendar["date"] = pd.to_datetime(calendar["date"])
calendar["mday"] = calendar["date"].dt.day.astype('int8')

In [ ]:
calendar['weekend'] = calendar['wday'].apply(lambda x: 1 if x>5 else 0).astype('int8')
calendar['year'] = (calendar['year']-calendar['year'].min()).astype('int8')
calendar['d'] = calendar['d'].str.replace('d_','').astype('int16')

In [ ]:
# def merge_by_concat(df1, df2, merge_on):
#     merged_gf = df1[merge_on]
#     merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
#     new_columns = [col for col in list(merged_gf) if col not in merge_on]
#     df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
#     return df1

In [ ]:
# prices['price_max'] = prices.groupby(['store_id','item_id'])['sell_price'].transform('max')
# prices['price_min'] = prices.groupby(['store_id','item_id'])['sell_price'].transform('min')
# prices['price_std'] = prices.groupby(['store_id','item_id'])['sell_price'].transform('std')
# prices['price_mean'] = prices.groupby(['store_id','item_id'])['sell_price'].transform('mean')

# # and do price normalization (min/max scaling)
# prices['price_norm'] = prices['sell_price']/prices['price_max']

# # Some items can be inflation dependent
# # and some items are very "stable"
# prices['price_nunique'] = prices.groupby(['store_id','item_id'])['sell_price'].transform('nunique')
# prices['item_nunique'] = prices.groupby(['store_id','sell_price'])['item_id'].transform('nunique')

# # I would like some "rolling" aggregations
# # but would like months and years as "window"
# calendar_prices = calendar[['wm_yr_wk','month','year']]
# calendar_prices = calendar_prices.drop_duplicates(subset=['wm_yr_wk'])
# prices = prices.merge(calendar_prices[['wm_yr_wk','month','year']], on=['wm_yr_wk'], how='left')
# del calendar_prices

# # Now we can add price "momentum" (some sort of)
# # Shifted by week 
# # by month mean
# # by year mean
# prices['price_momentum'] = prices['sell_price']/prices.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))
# prices['price_momentum_m'] = prices['sell_price']/prices.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
# prices['price_momentum_y'] = prices['sell_price']/prices.groupby(['store_id','item_id','year'])['sell_price'].transform('mean')

# del prices['month'], prices['year']

# release_df = prices.groupby(['store_id','item_id'])['wm_yr_wk'].agg(['min']).reset_index()
# release_df.columns = ['store_id','item_id','release']

# # Now we can merge release_df
# df_train = merge_by_concat(df_train, release_df, ['store_id','item_id'])
# del release_df

In [ ]:
# prices.info()
# prices.isna().sum()


In [ ]:
# prices.head()

In [ ]:
# prices['price_momentum'] = prices['price_momentum'].fillna(0)

In [ ]:
# prices.info()

In [ ]:
# df_train.info()

In [ ]:
# calendar.info()

In [ ]:
# df_train

In [ ]:
# df = df_train.melt(id_vars=["id",'item_id','dept_id','cat_id','store_id','state_id','release'], var_name="day", value_name="sales")
# del df_train
# df['day'] = df['day'].str.replace('d_','').astype('int16')

In [ ]:
# df.info()

In [ ]:
# df = df.merge(calendar, how='left', left_on='day', right_on='d' ).drop(['day'], axis = 1)
# df = df.merge(prices, how = "left", on = ["store_id", "item_id", "wm_yr_wk"])

In [ ]:
# df.head()

In [ ]:
# df_CA = df[df['state_id']=="CA"]

In [ ]:
# df_CA.info()
# del df

In [ ]:
# df_CA = df_CA.merge(calendar, how='left', left_on='day', right_on='d' ).drop(['day'], axis = 1)
# df_CA = df_CA.merge(prices, how = "left", on = ["store_id", "item_id", "wm_yr_wk"])

In [ ]:
df['cat_id'] = df['cat_id'].astype(str)
df['store_id'] = df['store_id'].astype(str)
df['dept_id'] = df['dept_id'].astype(str)
df['id'] = df['id'].astype(str)
df['state_id'] = df['state_id'].astype(str)

In [ ]:
df['cat_id']  = (df['store_id'] + '_' + df['cat_id'])
df['dept_id'] = (df['store_id'] + '_' + df['dept_id'])
df['id'] = (df['store_id'] + '_' + df['id'])

In [ ]:
day_cols = [col for col in df.columns if col.startswith('d_')]
idx = [int(col.split('d_')[1]) for col in day_cols]

# will be useful later
train_date_id = pd.to_datetime(calendar[calendar.d.isin(idx)].date)

In [ ]:
def transpose(column, index, day_col):
    """
    Turn the row oriented time series into a column oriented one 
    """
    ts = []
    new_cols = df[column].unique()
    
    for value in new_cols:
        value_ts = df[df[column] == value]
        vertical = value_ts[day_col].sum().T
        vertical.index = index
        ts.append(vertical)
    return pd.DataFrame({k: v for k, v in zip(new_cols, ts)})

In [ ]:
# state_t = transpose('state_id', train_date_id, day_cols)
# store_t = transpose('store_id', train_date_id, day_cols)
# cat_t = transpose('cat_id', train_date_id, day_cols)
# dept_t = transpose('dept_id', train_date_id, day_cols)
# item_t = transpose('id', train_date_id, day_cols)

# train_df = pd.concat([state_t, store_t, cat_t,dept_t, item_t], 1)

# # Total column is the root node -- the sum of of all demand across all stores (we have data on, at least)
# train_df['total'] = train_df['CA'] + train_df['TX'] + train_df['WI']


# del state_t
# del store_t
# del cat_t
# del dept_t
# del item_t

In [ ]:
# train_df.head()

In [ ]:
states = df["state_id"].unique()
stores = df["store_id"].unique()
categories = df['cat_id'].unique()
departments = df['dept_id'].unique()
products = df['id'].unique()
total = {'total': list(states)}
store = {k: [v for v in stores if v.startswith(k)] for k in states}
category = {k: [v for v in categories if v.startswith(k)] for k in stores}
department = {k: [v for v in departments if v.startswith(k)] for k in categories}
product = {k: [v for v in products if v.startswith(k)] for k in departments}

hierarchy = {**total, **store,**category, **department, **product}
del df

In [ ]:
del states, stores, categories, departments, products
del total, store, category, department, product

In [ ]:
# train_df.to_csv('train.csv')
train_df = pd.read_csv('./train.csv')

In [ ]:
from hts import HTSRegressor

clf = HTSRegressor(model='prophet', revision_method='OLS', n_jobs=12)
model = clf.fit(train_df, hierarchy)
preds = model.predict(steps_ahead=28)